# Using OpenAI models with Strands Agent

## Overview

Strands Agents is an SDK that takes a model-driven approach to building and running AI agents in just a few lines of code. Strands supports multiple providers and models hosted anywhere.

[LiteLLM](https://docs.litellm.ai/docs/) is a unified interface for various LLM providers that allows you to interact with models from Amazon, Anthropic, OpenAI, and many others through a single API. The Strands Agent SDK implements a LiteLLM provider, allowing you to run agents against any model LiteLLM supports.

In this example, we will show you how to use `gpt-4.0-mini` model as the underlying model in your Strands Agent. We will use a simple agent use case with a weather and a get time tool.


## Agent Details
<div style="float: left; margin-right: 20px;">
    
|Feature             |Description                                        |
|--------------------|---------------------------------------------------|
|Feature used        |LiteLLM model                                      |
|Agent Structure     |Single agent architecture                          |

</div>

## Architecture

<div style="text-align:center">
    <img src="images/architecture.png" width="65%" />
</div>

## Key Features
* **LiteLLM model**: using a model provided via LiteLLM

## Setup and prerequisites

### Prerequisites
* Python 3.10+
* Open AI APi Key
* gpt-4.0-mini access

Let's now install the requirement packages for our Strands Agent

In [2]:
# installing pre-requisites
!uv pip install -r requirements.txt

Using Python 3.13.5 environment at: e:\OneDrive\OneDriveOnitbuddy\OneDrive\workbench\exampletest\agenticAIeducation\.venv
Resolved 95 packages in 2.10s
Uninstalled 2 packages in 46ms
         If the cache and target directories are on different filesystems, hardlinking may not be supported.
         If this is intentional, set `export UV_LINK_MODE=copy` or use `--link-mode=copy` to suppress this warning.
Installed 4 packages in 1.83s
 - aiohttp==3.11.18
 + aiohttp==3.12.15
 - aiosignal==1.3.2
 + aiosignal==1.4.0
 + litellm==1.72.9
 + tokenizers==0.21.4


### Importing dependency packages

Now let's import the dependency packages

In [3]:
import os
from datetime import datetime
from datetime import timezone as tz
from typing import Any
from zoneinfo import ZoneInfo

from strands import Agent, tool
from strands.models.litellm import LiteLLMModel
from dotenv import load_dotenv

### Importing your .env variables

This will set up the OpenAPI Key

In [4]:
load_dotenv(override=True)

True

In [5]:
# Set up your OpenAI API key
# Method 1: Set environment variable directly
import os
OPEN_AI_KEY = os.getenv("OPENAI_API_KEY")  # Replace with your actual key


# Method 2: Load from .env file (recommended for security)
# Create a .env file with: OPENAI_API_KEY=your_actual_key_here
# Then the load_dotenv() call below will load it


### Setting up custom tools

Let's now setup two dummy tools to test our agent

In [6]:
@tool
def current_time(timezone: str = "UTC") -> str:
    if timezone.upper() == "UTC":
        timezone_obj: Any = tz.utc
    else:
        timezone_obj = ZoneInfo(timezone)

    return datetime.now(timezone_obj).isoformat()


@tool
def current_weather(city: str) -> str:
    # Dummy implementation. Please replace with actual weather API call.
    return "sunny"

### Defining agent underlying LLM model

Next let's define our agent underlying model using LiteLLM. We will set it to `gpt-4.1-mini`

In [7]:
# Set your OpenAI API key in environment variable
# os.environ["OPENAI_API_KEY"] = "your_openai_api_key_here"  # Uncomment and add your key

# Create LiteLLM model with string model identifier (not a function!)
litellm_model = LiteLLMModel(
    model_id="gpt-4o-mini",  # Pass the model name as a string
    params={"max_tokens": 16383, "temperature": 0.7}
)

### Defining Agent

Now that we have all the required information available, let's define our agent

In [8]:
system_prompt = "You are a simple agent that can tell the time and the weather"
agent = Agent(
    model=litellm_model,
    system_prompt=system_prompt,
    tools=[current_time, current_weather],
)

### Testing agent

Let's now invoke the agent to test it

In [9]:
results = agent("What time is it in Seattle? And how is the weather?")


Tool #1: current_time

Tool #2: current_weather
The current time in Seattle is 7:03 PM, and the weather is sunny.

#### Analysing the agent's results

Nice! We've invoked our agent for the first time! Let's now explore the results object. First thing we can see is the messages being exchanged by the agent in the agent's object

In [10]:
agent.messages

[{'role': 'user',
  'content': [{'text': 'What time is it in Seattle? And how is the weather?'}]},
 {'role': 'assistant',
  'content': [{'toolUse': {'toolUseId': 'call_khJ2LKZdQWCqCEUwQiVCNgk5',
     'name': 'current_time',
     'input': {'timezone': 'America/Los_Angeles'}}},
   {'toolUse': {'toolUseId': 'call_5cjpKEzXsQvRqIQoAkMaGl1D',
     'name': 'current_weather',
     'input': {'city': 'Seattle'}}}]},
 {'role': 'user',
  'content': [{'toolResult': {'toolUseId': 'call_khJ2LKZdQWCqCEUwQiVCNgk5',
     'status': 'success',
     'content': [{'text': '2025-08-07T19:03:46.939818-07:00'}]}},
   {'toolResult': {'toolUseId': 'call_5cjpKEzXsQvRqIQoAkMaGl1D',
     'status': 'success',
     'content': [{'text': 'sunny'}]}}]},
 {'role': 'assistant',
  'content': [{'text': 'The current time in Seattle is 7:03 PM, and the weather is sunny.'}]}]

Next we can take a look at the usage of our agent for the last query by analysing the result `metrics`

In [11]:
results.metrics

EventLoopMetrics(cycle_count=2, tool_metrics={'current_weather': ToolMetrics(tool={'toolUseId': 'call_5cjpKEzXsQvRqIQoAkMaGl1D', 'name': 'current_weather', 'input': {'city': 'Seattle'}}, call_count=1, success_count=1, error_count=0, total_time=0.0009012222290039062), 'current_time': ToolMetrics(tool={'toolUseId': 'call_khJ2LKZdQWCqCEUwQiVCNgk5', 'name': 'current_time', 'input': {'timezone': 'America/Los_Angeles'}}, call_count=1, success_count=1, error_count=0, total_time=0.0526118278503418)}, cycle_durations=[1.0967774391174316], traces=[<strands.telemetry.metrics.Trace object at 0x000001C4853F23C0>, <strands.telemetry.metrics.Trace object at 0x000001C4FDF98DD0>], accumulated_usage={'inputTokens': 279, 'outputTokens': 67, 'totalTokens': 346}, accumulated_metrics={'latencyMs': 0})

### Congratulations!

In this notebook you learned how to use LiteLLM with OpeanAi serving answers for weather agent.